In [1]:
import import_ipynb
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import bertkeras as bk
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from unidecode import unidecode
from scipy.stats import norm, boxcox
pd.options.display.max_colwidth = 1000
sns.set()

importing Jupyter notebook from bertkeras.ipynb


In [2]:
df= pd.read_csv('../dataset/training.csv', index_col='ID')
labels = df['label']
df = df.drop(columns=[ '﻿ASIN','parent_asin','Product Group Description'])
df = df.fillna("")

cols = df.columns.tolist()
cols.remove('label')
text = df[cols].apply(' '.join, axis=1)


In [3]:
html_re = re.compile('<[^>]*>')
alphanum_re = re.compile('[^0-9a-z]+') 
weight_re = re.compile('[\d.]+[ ]?(kg+|lbs+|g+|lb+)')

# check if it actually helps, tests https://regex101.com/r/xwnV5l/1
size_re = re.compile('[\d.]+([ cminch]+)?[ x*]+[\d.]+([ cminch]+)?([ x*]+[\d.]+)?([ cminch]+)')

def html_cleanup(html):
    return re.sub(html_re, ' ', html)
def lower_case(txt):
    return txt.lower()
def split_by_non_alphanumeric(txt):
    return re.sub(alphanum_re, ' ', txt)
def decode_chars(txt):
    return unidecode(txt)

def cleanup_size(txt):
    return re.sub(size_re, ' ', txt).replace('size:', '')

def cleanup_weight(txt):
    return re.sub(weight_re, ' ', txt)
    
def cleanup(text):
    res = text.apply(html_cleanup).apply(decode_chars).apply(lower_case) \
              .apply(split_by_non_alphanumeric).apply(cleanup_size) \
              .apply(cleanup_weight)
    return res

In [4]:
texts = cleanup(text)

In [5]:
df = pd.DataFrame( {"texts" :texts, "labels": labels} )

In [6]:
df

,texts,labels
ID,,
1686,regalo divertido de la taza del cafe el mejor regalo para los regalos del dia de padre de los papas de su papa mas afortunado de la hijo del mundo absolutely love your order or contact us and we ll make it right your satisfaction is uaranteed high quality mug makes the perfect gift for everyone printed on only the highest quality mugs the print will never fade no matter how many times it is washed packaged and shipped from the usa dishwasher and microwave safe shipped in a custom made styrofoam package to ensure it arrives perfect guaranteed,0
4401,neonblond cookie box vi tkovice ski resort czech republic ski resort christmas metal container description there s no more heartfelt gift this holiday season like the gift of freshly baked christmas cookies give those precious gifts this year in beautiful christmas wreath cookie tin that will be treasured for years to come fill them with home baked or store bought goodies and stand out from the holiday party gifts these christmas cookie tins are certified food safe and just the perfect size to be used long after those yummy goodies are gone beautiful christmas cookie tin for gifts storage more neonblond s stylish stainless steel cookie tins size with a one of a kind design perfect for gifting homemade cookies and treats during the holidays or at any time of year multiple uses around the home these attractive tins are perfect for food storage craft supplies storage holiday cookie gifts children s activities and much more the nesting design means compact storage when not in use qua...,0
7340,egg massager remote control vi 10 speed handheld massagers item type massagers brand name xc ushio size aterial plastic commodity quality certification ce model number 4000004366 suggestive no type picture no material2 plastic abs remote control distance 5m unit type set color purple pink package size package weight 0 0 egg massager remote control vi 10 speed massager massager massager material plastic abs color purple pink battery aaa battery not included massager 10 speeds egg remote control distance 5m note we will remain anonymous and shipping package without item information please feel free to purchase it package size size package weight 0 0 color purple pink material2 plastic abs,1
3228,massager charging waterproof vi ring massagers handheld item type massagers brand name prettylove commodity quality certification ce powered usb rechargeable material function function s model number 210152 waterproof 100 waterproof unit type piece suggestive no type function 3 massager package discreet package picture no size as shown noise low noise design color black purple package size function 2 vi ring package weight 0 0 package size function 3 massager function function s function 2 vi ring powered usb rechargeable,1
2148,art oil painting reproduction size painting name flight of the turks by diaz de la pena narcissevirgile rtgallery com is a registered australian company so purchases can be made with confidence when art is purchased from rtgallery com an artist is commissioned to paint the artwork to any specified size you are currently looking at a painting called flight of the turks by diaz de la pena narcissevirgile 100 hand painted life time guarantee size 24 by 36 inch arrives with tnt to your door unframed oil paintings fine art oil paintings home decore flight of the turks diaz de la pena narcissevirgile,0
196,optisex stunning steven vibrating 7 inch cock and balls with suction natural flesh the advanced compact steamer this handheld steamer has more steaming power than you can imagine travel steamer requires only a minute icare menstrual cups and a half to reach the needed temperature and has enough water for up to 10 minutes of non stop steaming iron steamer goes beyond comparison with the conventional ironing the dry steam goes through the fabric and tackles the creases and destroying all bacteria along the way clothes steamer for quicker ironing ironing with this po

In [7]:
txt_train, txt_test, lbl_train, lbl_test = train_test_split(
    texts, labels, test_size=0.001, random_state=42)

In [8]:
tokenizer = bk.create_tokenizer_from_hub_module(vocab_file='vocab.txt')
max_seq_length = 512

using vocab file from vocab.txt


In [9]:
# Convert data to InputExample format
train_examples = bk.convert_text_to_examples(txt_train, lbl_train.values)
test_examples = bk.convert_text_to_examples(txt_test, lbl_test.values)

In [10]:

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = bk.convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = bk.convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [11]:
train_examples[0].__dict__

{'guid': None,
 'text_a': 'round emerald 14k rose gold engagement ring aeur rosemary ring a classic basket setting holds an 7mm round cut center stone flanked by two 5mm round cut accent stones in this sparkling ring carefully handcrafted in your choice of gems and metals other products ',
 'text_b': None,
 'label': 0}

In [12]:
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = bk.BertLayer(n_fine_tune_layers=2, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [13]:
ACCURACY_THRESHOLD = 0.999
LOSS_THRESHOLD = 0.04
class myCallback(tf.keras.callbacks.Callback): 
    def on_batch_end(self, batch, logs): 
        if(logs.get('acc') > ACCURACY_THRESHOLD and logs.get('loss')<LOSS_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100)) 
          self.model.stop_training = True

In [14]:
model = build_model(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 512)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [15]:
# Initialize session
sess = tf.Session()
# Instantiate variables
bk.initialize_vars(sess)
tb = tf.compat.v1.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=24, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')
stop999 = myCallback()
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    callbacks = [ tb], #, stop999
    epochs=4,
    batch_size=24
)

Train on 5480 samples, validate on 6 samples
Epoch 1/4
5480/5480 [==============================] - 267s 49ms/sample - loss: 0.2954 - acc: 0.8708 - val_loss: 0.0174 - val_acc: 1.0000
Epoch 2/4
5480/5480 [==============================] - 264s 48ms/sample - loss: 0.1620 - acc: 0.9535 - val_loss: 0.0060 - val_acc: 1.0000
Epoch 3/4
5480/5480 [==============================] - 264s 48ms/sample - loss: 0.0874 - acc: 0.9757 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 4/4
5480/5480 [==============================] - 264s 48ms/sample - loss: 0.0545 - acc: 0.9854 - val_loss: 0.0012 - val_acc: 1.0000


In [16]:
model.save('restricted.h5')

In [17]:
df_subm= pd.read_csv('../dataset/public_test_features.csv', index_col='ID')
df_subm = df_subm.drop(columns=[ '﻿ASIN','parent_asin','Product Group Description'])
df_subm = df_subm.fillna("")

text_subm = df_subm[cols].apply(' '.join, axis=1)


In [18]:
texts_subm = cleanup(text_subm)

In [19]:
subm_examples = bk.convert_text_to_examples(texts_subm, [0]*len(texts_subm))

In [20]:
(subm_input_ids, subm_input_masks, subm_segment_ids, subm_labels
) = bk.convert_examples_to_features(tokenizer, subm_examples, max_seq_length=max_seq_length)

In [21]:
subm_examples[0].text_a

'vintage mama e hija amor corazon cadena collar mujeres dia de la madre regalo especificaciones disea o de mama e hija colgante con forma de coraza3n hecho de material de aleacia3n de alta calidad con mano de obra fina regalo perfecto para el daa de la madre u otras ocasiones tipo collar ganero mujeres tema estilo vintage material aleacia3n de ocasiones fiesta daa de la madre regalo vida diaria etc caracterasticas colgante de coraza3n patra3n de letras regalo para madre longitud de la cadena  69 pulgadas  pulgadas aprox notas debido a la diferencia de configuracia3n de la luz y la pantalla el color del artaculo puede ser ligeramente diferente al de las ima genes puede haber una ligera diferencia de dimensia3n debido a las diferentes medidas manuales el paquete incluye 1 x collar item specifics fino o de moda moda tipo de articulo collares nombre de la marca bluelans estilo trendy forma patron carta tipo de cadena cadena de eslabones tamano colgante 5cm compatibilidad totalmente compati

In [22]:
testCount = len(subm_input_ids)
def pickMax(arr):
  m = max(arr)
  return [x for x in arr]

threshold = 0.5
def tolabel(arr):
  # return int(max(arr)>=threshold)
  return max(arr)
    


In [23]:
predictions = model.predict([subm_input_ids[0:testCount], 
                                subm_input_masks[0:testCount], 
                                subm_segment_ids[0:testCount]])

In [24]:
pred = [ tolabel(p) for p in predictions]

In [25]:
results = pd.DataFrame({"text":texts_subm, "label":pred})

In [26]:
results[results['label']>0.05][results['label']<0.9]


/home/drmegavolt/miniconda3/envs/restricted_products/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,text,label
ID,,
1639,smilinggirl car boot organizer car boot bag waterproof foldable storage box can accommodate 66l for car suv minivan truck indoor uses red product specifications name trunk storage box style folding material high density 1680d waterproof oxford cloth soft and durable size volume 66l features 1 special design used in the trunk of the car so that your car will not get dirty it retains the health and order of the car 2 easy to install just unfold the finisher slide it into the hardened bottom plate connect the dividers and fill in any items you like 3 folding design you can fold it into small sizes to save space when not in use store and organize your car accessories and other everyday needs in an orderly manner 4 multi functional design some small outer bags can be used to store some small items have enough pockets and compartments to help you stay in the right place 5 wide use suitable for any type of vehicle and any condition used to organize emergency equipment tools groceries to...,0.219610
1681,mary kay mk signaturei waterproof mascara black 33 fl oz featuring smart wiper technology body scrubs mary kay mk skin care products for women skin care for women skin care for men body scrubs for women,0.440340
2021,sxzhsmy001 usb charging intelligent heating 45 deg 12 frequency vibrating bar suitable for female toys telescopic rabbit vibrator notice this product is a personal use item for your health it is recommended not to mix it with others please clean it before and after using it wash with soap and water after use let it dry completely before storage number of products 1 product material silica gel product size 23cm long and diameter product color rose red purple control switch button control charging mode usb charging function mode intelligent heating 12 frequency vibration telescopic maximum noise 40 decibels waterproof 100 waterproof tip only toys no other products due to the problem of light photography there may be some chromatic aberration since the size is measured manually there may be some errors that are normal the color of the product will vary slightly depending on the setting of the computer screen the actual product color will prevail please forgive me thank you we will al...,0.893100
6006,vaginalnel vibrator silicone ben wa balls kegel weights vaginalne tighten exercise trainer dumbbell bead toy bladder control devices for women why ladies need to do kegel exercise 1 heavy kegel balls exercise can contract the vaginalne and make female be more confident 2 after childbirth pelvic floor muscle exercises could strengthen the muscles that support the bladder uterus and large intestine avoid the leakage of urine specification material medical silicone size weight color purple rosy black waterproof yes tips for clean store please clean it with warm and wet towel before and after you use it dry it with your towel for convenient it should be placed and stored at clean places and please avoid sunlight directly safety concern do not use boiled water to clean it and rinse it under the tap directly it is recommended that you use it with water based lubricant instead of silicone based lubricant tips for pregnant women please keep in mind that you must consult with your doctor...,0.068476
797,activscience neck firming cream anti aging moisturizer for neck decollete double chin reducer skin tightening cream day night for men women 2 fl oz for best results apply activ neck firming cream times daily to the contours of the chin jawline neck and decolletage massage in a circular motion while making sure to apply to the back of the neck as well the skin below our chin is just as delicate as the skin above it when applied to the neck and the lower face activ neck firming cream refines facial contours and brightens the complexion leaving skin radiant don t let sagging skin get in the way of your natural beauty activscience brings the most advanced science to skin care we take an evidence based approach to formula

In [27]:
results.to_csv('results.csv', columns=[ 'label'])